In [ ]:
#with datafrane
import pandas as pd
import tensorflow as tf

colnames = ['source', 'tar', 'gib']
data_df = pd.read_csv('/content/trans.txt', sep='\t', names=colnames)
final_df = data_df[['source', 'tar']]
print(final_df.head(1))

data_df = data_df[['source', 'target']]

source_sentences = []
target_sentences = []

for index, row in data_df.iterrows():
  src, tar = row[0], row[1]
  source_sentences.append(src)
  target_sentences.append(tar)

  source    tar
0    Hi.  سلام۔


In [ ]:
print(final_df.head(112))

             source                      tar
0               Hi.                    سلام۔
1             Help!                     مدد۔
2           Thanks.                   شکریہ۔
3           We won.              ہم جیت گئے۔
4          Beat it.               بھاگ جائو۔
..              ...                      ...
107  It won't work.      یہ کام نہیں کرے گا۔
108  Tom hates you.  ٹام تم سے نفرت کرتا ہے۔
109  Tom hates you.  ٹام آپ سے نفرت کرتا ہے۔
110  Tom's strange.             ٹام عجیب ہے۔
111  Tom's trapped.        ٹام پھنسا ہوا ہے۔

[112 rows x 2 columns]


In [ ]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = '/content/trans.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('.') for sentence in  raw_data]


In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', 
                                                        string_punctuation))
   
    return clean_sentence

In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
# Clean sentences
#english_sentences = [clean_sentence(pair) for pair in source_sentences]
#urdu_sentences = [clean_sentence(peir) for peir in target_sentences]

english_sentences = source_sentences
urdu_sentences = target_sentences

# Tokenize words
urd_text_tokenized, urd_text_tokenizer = tokenize(urdu_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length urdu sentence: {}'.format(len(max(urd_text_tokenized,
                                                        key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,
                                                           key=len))))


# Check language length
urdu_vocab = len(urd_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("urdu vocabulary is of {} unique words".format(urdu_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))


Maximum length urdu sentence: 21
Maximum length english sentence: 18
urdu vocabulary is of 1696 unique words
English vocabulary is of 1326 unique words


In [ ]:
max_urdu_len = int(len(max(urd_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

urd_pad_sentence = pad_sequences(urd_text_tokenized, max_urdu_len,
                                 padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, 
                                 padding = "post")

# Reshape data
urd_pad_sentence = urd_pad_sentence.reshape(*urd_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [ ]:
input_sequence = Input(shape=(max_english_len,))
embedding = Embedding(input_dim=english_vocab, output_dim=128,) (input_sequence)
encoder = LSTM(64, return_sequences=False)                   (embedding)
r_vec = RepeatVector(max_urdu_len)                        (encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)       (r_vec)
logits = TimeDistributed(Dense(urdu_vocab))               (decoder)


In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')  (logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 18)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 18, 128)           169728    
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_2 (RepeatVect  (None, 21, 64)           0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 21, 64)            33024     
                                                                 
 time_distributed_2 (TimeDis  (None, 21, 1696)         110240    
 tributed)                                                 

In [ ]:
model_results = enc_dec_model.fit(eng_pad_sentence, 
                                  urd_pad_sentence, 
                                  batch_size=15, epochs=100)


Epoch 1/100
77/77 [==============================] - 3s 37ms/step - loss: 0.5178 - accuracy: 0.8684
Epoch 2/100
77/77 [==============================] - 3s 37ms/step - loss: 0.5127 - accuracy: 0.8706
Epoch 3/100
77/77 [==============================] - 3s 38ms/step - loss: 0.5127 - accuracy: 0.8682
Epoch 4/100
77/77 [==============================] - 3s 37ms/step - loss: 0.5128 - accuracy: 0.8673
Epoch 5/100
77/77 [==============================] - 3s 38ms/step - loss: 0.5072 - accuracy: 0.8691
Epoch 6/100
77/77 [==============================] - 3s 38ms/step - loss: 0.5110 - accuracy: 0.8668
Epoch 7/100
77/77 [==============================] - 3s 37ms/step - loss: 0.5073 - accuracy: 0.8701
Epoch 8/100
77/77 [==============================] - 3s 38ms/step - loss: 0.5105 - accuracy: 0.8669
Epoch 9/100
77/77 [==============================] - 3s 38ms/step - loss: 0.5042 - accuracy: 0.8695
Epoch 10/100
77/77 [==============================] - 3s 37ms/step - loss: 0.4955 - accuracy: 0.8731

In [ ]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(
        logits, 1)])

index = 1000
print("The english sentence is: {}".format(english_sentences[index]))
print("The urdu sentence is: {}".format(urdu_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(eng_pad_sentence[
                                    index:index+1])[0], urd_text_tokenizer))

The english sentence is: Five years have gone by since my father died.
The urdu sentence is: میرے ابو کو فوت ہوئے پانچ سال ہو گئے ہے۔
The predicted sentence is :
میرے ابو کو گزرے ہوئے پانچ سال ہو گئے ہے۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>


In [ ]:
input = "Design thinking is a five step process to come up with meaningful ideas. That solve real problems. For a particular group of people. The process is taught in top design and business schools around the world. It has brought many businesses lots of happy customers."

In [ ]:

input_arr = input.split('.')
print(input_arr)



['Design thinking is a five step process to come up with meaningful ideas', ' That solve real problems', ' For a particular group of people', ' The process is taught in top design and business schools around the world', ' It has brought many businesses lots of happy customers', '']


In [ ]:
lent = len(input_arr)
for i in range(lent):
  index = i
  input_sentence_tokenized, input_sentence_tokenizer = tokenize(input_arr)
  input_vocab = len(input_sentence_tokenizer.word_index) + 1
  max_input_len = int(len(max(input_sentence_tokenized,key=len)))

  input_pad_sentence = pad_sequences(input_sentence_tokenized, max_input_len,
                                  padding = "post")

  input_pad_sentence = input_pad_sentence.reshape(*input_pad_sentence.shape, 1)
  print(logits_to_sentence(enc_dec_model.predict(input_pad_sentence[
                                      index:index+1])[0], urd_text_tokenizer))

تم اس کو میں سے فرانسیسی کر کر سکتا ہے۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
مجھے صرف پوری دفعہ نیں سکتا۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
اس نے فطرت میں کوئی بعد، نظر نہیں نہیں ہے۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
میں نے سے میں کو میں کس بھی لیئے سکتا تھا۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
ہم رات پرانے کو مر رہی ہیں۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
ٹام کمزور ہے۔ <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
